# Rating Products

Bir ürünü satın aldıran iki büyük fenomen vardır bunlar Sosyal İspat (Social Proof) ve Kalabalıkların Bilgeliğidir (The Wisdom of Crowds). Dolayısıyla ürünlerin puanlanmasına ve yorumlanmasına ihtiyaç vardır ki satış için aradan sıyrılabilsin. Kullanıcı açısından bir şey alırken amaç fiyat performans açısından en iyi ürüne ulaşmaktır. Pazar yerlerinin görevi ise kullanıcılara en doğru ürünleri ulaştırmaktır. Bu sebeple pazar yerleri açısından ürünleri manipule etmeden, ürünlerin puanlarının hesaplanması ve ürün yorumlarının sıralanmasını gerektirir. Bu konular ise geliştiricileri aşağıdaki başlıklar açısından ilgilendirmektedir;

* Ürün puanlarının hesaplanması
* Ürünlerin sıralanması
* Ürün detay sayfalarındaki kullanıcı yorumlarının sıralanması
* Sayfa, süreç ve etkileșim alanlarının tasarımları
* Özellik denemeleri
* Olası aksiyonların ve reaksiyonların test edilmesi


In [6]:
# Bu bölümde olası faktörleri göz önünde bulundurarak ağırlıklı ürün puanlama işlemleri gerçekleştirilecektir.

import pandas as pd 
import math
import scipy.stats as st
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# pd.set_option('display.max_coulmns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.width', 500)
# pd.set_option('display.expand_frame_repr',False)
# pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [11]:
# Course Name: (50+ Saat) Python A-Z™: Veri Bilimi ve Machine Learning
# Rating: 4.8 (4.764925)
# Total Rating: 4611
# Percentage of Ratings: 75, 20, 4, 1, <1
# Approximate Numerical Equivalents: 3458, 922, 184, 46, 6

# Yukarıdaki Udemy Kursuna ilişkin verilerin analizi ile rating yapacağız.
df = pd.read_csv("dataset/course_reviews.csv")
df.head()

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered
0,5.0,2021-02-05 07:45:55,2021-01-25 15:12:08,5.0,0.0,0.0
1,5.0,2021-02-04 21:05:32,2021-02-04 20:43:40,1.0,0.0,0.0
2,4.5,2021-02-04 20:34:03,2019-07-04 23:23:27,1.0,0.0,0.0
3,5.0,2021-02-04 16:56:28,2021-02-04 14:41:29,10.0,0.0,0.0
4,4.0,2021-02-04 15:00:24,2020-10-13 03:10:07,10.0,0.0,0.0


In [13]:
# Ratinglerin dağılımına gitmek istersek;
df["Rating"].value_counts()
#5 puandan 3267 adet, 4.5 puandan 475 adet olduğunu görmekteyiz. 

5.0    3267
4.5     475
4.0     383
3.5      96
3.0      62
1.0      15
2.0      12
2.5      11
1.5       2
Name: Rating, dtype: int64

In [14]:
# Sorulan sorular değişkenini analiz etmek için
df["Questions Asked"].value_counts()

0.0     3867
1.0      276
2.0       80
3.0       43
4.0       15
5.0       13
6.0        9
8.0        5
9.0        3
14.0       2
11.0       2
7.0        2
10.0       2
15.0       2
22.0       1
12.0       1
Name: Questions Asked, dtype: int64

In [17]:
# Sorulan soru kırılımında verilen puanlara bakmak istersek
#Örneğin 2 soru soran 80 kişi ortalama 4.806250 puan vermiş, hiç soru sormayan 3867 kişinin puan ortalaması 4.765193'dir.

df.groupby("Questions Asked").agg({"Questions Asked": "count",
                                   "Rating": "mean"})

,Questions Asked,Rating
Questions Asked,,
0.0,3867,4.765193
1.0,276,4.740942
2.0,80,4.806250
3.0,43,4.744186
4.0,15,4.833333
5.0,13,4.653846
6.0,9,5.000000
7.0,2,4.750000
8.0,5,4.900000


In [18]:
#AVERAGE
#ORTALAMA PUAN

# Sadece ortalamaya göre rating hesaplamak istersek;

df["Rating"].mean()
# Kurs puanı ise 4.7649'dan 4.8'e yuvarlanmıştır.
# Bu bir ortalama hesabı için kullanılabilir. Ancak sadece ortalamya göre rating hesaplandığında son zamanlardaki trendleri kaçırmış oluruz.
# İlk 3 ayda iyi bir hizmet sunulmuş daha sonrasında hizmetin bırakılması gibi bir case de sadece ortalama hesaplamak güncel trendleri kaçırmamıza sebep olur.


4.764284061993986

### Time-Based Weighted Average (Puan Zamanlarına Göre Ağırlıklı Ortalama)

In [19]:
df.head()

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered
0,5.0,2021-02-05 07:45:55,2021-01-25 15:12:08,5.0,0.0,0.0
1,5.0,2021-02-04 21:05:32,2021-02-04 20:43:40,1.0,0.0,0.0
2,4.5,2021-02-04 20:34:03,2019-07-04 23:23:27,1.0,0.0,0.0
3,5.0,2021-02-04 16:56:28,2021-02-04 14:41:29,10.0,0.0,0.0
4,4.0,2021-02-04 15:00:24,2020-10-13 03:10:07,10.0,0.0,0.0


In [22]:
df.info()
#Burada timestap değişkenimiz object bunu zaman değişkenine çevirmemiz gerekmektedir. 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4323 entries, 0 to 4322
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Rating              4323 non-null   float64
 1   Timestamp           4323 non-null   object 
 2   Enrolled            4323 non-null   object 
 3   Progress            4323 non-null   float64
 4   Questions Asked     4323 non-null   float64
 5   Questions Answered  4323 non-null   float64
dtypes: float64(4), object(2)
memory usage: 202.8+ KB


In [23]:
df["Timestamp"] = pd.to_datetime(df["Timestamp"])

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4323 entries, 0 to 4322
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Rating              4323 non-null   float64       
 1   Timestamp           4323 non-null   datetime64[ns]
 2   Enrolled            4323 non-null   object        
 3   Progress            4323 non-null   float64       
 4   Questions Asked     4323 non-null   float64       
 5   Questions Answered  4323 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 202.8+ KB


In [25]:
# Bunun yanı sıra yapılan yorumların ne kadar süre önce yapıldığını gün cinsinden ifade etmemiz gerekmektedir.
# Bu sebeple bir tarih belirleyip aradaki farkı bulmamız gerekmektedir.
current_date=pd.to_datetime("2021-02-10 0:0:0")
current_date

Timestamp('2021-02-10 00:00:00')

In [27]:
# Aradaki farkı hesaplayacağız,
df["days"]=(current_date-df["Timestamp"]).dt.days
df


,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered,days
0,5.0,2021-02-05 07:45:55,2021-01-25 15:12:08,5.0,0.0,0.0,4
1,5.0,2021-02-04 21:05:32,2021-02-04 20:43:40,1.0,0.0,0.0,5
2,4.5,2021-02-04 20:34:03,2019-07-04 23:23:27,1.0,0.0,0.0,5
3,5.0,2021-02-04 16:56:28,2021-02-04 14:41:29,10.0,0.0,0.0,5
4,4.0,2021-02-04 15:00:24,2020-10-13 03:10:07,10.0,0.0,0.0,5
...,...,...,...,...,...,...,...
4318,5.0,2019-05-17 09:51:44,2019-05-17 09:08:53,34.0,1.0,0.0,634
4319,5.0,2019-05-16 21:27:05,2019-05-16 20:32:15,5.0,0.0,0.0,635
4320,5.0,2019-05-16 20:22:26,2019-05-16 20:21:19,1.0,0.0,0.0,635
4321,5.0,2019-05-16 19:49:07,2019-05-16 19:47:29,1.0,0.0,0.0,635


In [35]:
#son 30 günde yapılan yorumlara nasıl erişiriz;
df[df["days"]<=30].count()
#toplam 194 tane yorum yapılmış son 30 dünde

Rating                194
Timestamp             194
Enrolled              194
Progress              194
Questions Asked       194
Questions Answered    194
days                  194
dtype: int64

In [37]:
# Son 30 günde yapılan ratinglerin ortalamasına bakmak istersek;

df.loc[df["days"]<=30, "Rating"].mean()

4.775773195876289

In [38]:
# Gün olarak 30'dan büyük 90'dan küçük eşit olan ratinglerin ortalamasına bakmak istersek;

df.loc[(df["days"]>30) & (df["days"]<=90) , "Rating"].mean()

4.763833992094861

In [41]:
#diğer tarihlere bakalım;
df.loc[(df["days"]>90) & (df["days"]<=180) , "Rating"].mean()


4.752503576537912

In [42]:
# Son koşulumuz da;
df.loc[df["days"]>180, "Rating"].mean()


4.76641586867305

In [51]:
# Yukardaki durumlar çerçevesinde kurs memnuniyeti konusunda son dönemlerde artış olduğu gözükmektedir. 
# Bu durumda zamana bağlı ağırlıklı ortalama ile hasaplama yapmak istersek, yukarıdaki her bir değere bir ağırlık vermemiz gerekecek ve daha yakın zamanı daha da fazla ağırlıklandıracağız.

df.loc[df["days"] <= 30, "Rating"].mean() * 28/100 + \
    df.loc[(df["days"] > 30) & (df["days"] <= 90), "Rating"].mean() * 26/100 + \
    df.loc[(df["days"] > 90) & (df["days"] <= 180), "Rating"].mean() * 24/100 + \
    df.loc[(df["days"] > 180), "Rating"].mean() * 22/100
    
# Yukarıda ters slash kullanılmasının sebebi ise bir alt satırda işleme devam edeceğimizi belirtiyoruz.
# Bunun yerine işlemi parantez içerisine de alabilidrdik.

4.765025682267194

In [52]:
def time_based_weighted_average(dataframe, w1=28, w2=26, w3=24, w4=22):
    return dataframe.loc[df["days"] <= 30, "Rating"].mean() * w1 / 100 + \
           dataframe.loc[(dataframe["days"] > 30) & (dataframe["days"] <= 90), "Rating"].mean() * w2 / 100 + \
           dataframe.loc[(dataframe["days"] > 90) & (dataframe["days"] <= 180), "Rating"].mean() * w3 / 100 + \
           dataframe.loc[(dataframe["days"] > 180), "Rating"].mean() * w4 / 100

time_based_weighted_average(df)

time_based_weighted_average(df, 30, 26, 22, 22)



4.765491074653962

### Kullanıcı Temelli Ağırlıklı Ortalama ( User-Based Weighted Average )

- Bazı kullanıcıların çok düşük oranlarda izlediğini görmekteyiz, bunlar ile çok fazla izleyenler arasında ağırlığın değişmesi gerekir. 
- Bu duruma user-based ya da user-quality denir. 


In [53]:
df.head()

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered,days
0,5.0,2021-02-05 07:45:55,2021-01-25 15:12:08,5.0,0.0,0.0,4
1,5.0,2021-02-04 21:05:32,2021-02-04 20:43:40,1.0,0.0,0.0,5
2,4.5,2021-02-04 20:34:03,2019-07-04 23:23:27,1.0,0.0,0.0,5
3,5.0,2021-02-04 16:56:28,2021-02-04 14:41:29,10.0,0.0,0.0,5
4,4.0,2021-02-04 15:00:24,2020-10-13 03:10:07,10.0,0.0,0.0,5


In [55]:
df.groupby("Progress").agg({"Rating": "mean"})
#Buradan kursun %1ini izlemiş kişilerin ortalam 4.64 puan verdiğini
#Kursun %97-98ini izlemiş kişilerin 5 puan verdiği ve kursun tamamını izlemiş kişilerin 4.86 puan verdiğini görmekteyiz. 
#Dolayısıyla kursu izleme oranı ile puan verme arasında bir ilişki olduğunu söyleyebiliriz. 

,Rating
Progress,
0.0,4.673913
1.0,4.642691
2.0,4.654762
3.0,4.663551
4.0,4.777328
...,...
94.0,5.000000
95.0,4.794118
97.0,5.000000


In [60]:
df.loc[df["Progress"] <= 10, "Rating"].mean() * 22 / 100 + \
    df.loc[(df["Progress"] > 10) & (df["Progress"] <= 45), "Rating"].mean() * 24/100 + \
    df.loc[(df["Progress"] > 45) & (df["Progress"] <=75), "Rating"].mean() * 26/100 + \
    df.loc[df["Progress"] >75, "Rating"].mean() * 28/100
# Yukarda bahsettiğimiz ağırlıkları burada uyguladık. 
# Yani kursu tamamlamış bir kişinin puanı ve hiç kullanmamış birisinin puanı aynı olmamalıdır. 
# Yukarıda "user based" ve "user quality" terimlerinden bahsetmiştik. Buradaki kullanıcı kalitesini etkileyen metrik kursu tamamlama oranıdır. 

4.800257704672543

In [63]:
#Fonksiyonunu yazalım, 
def user_based_weighted_average(dataframe, w1=22, w2=24, w3=26, w4=28):
    return dataframe.loc[dataframe["Progress"] <= 10, "Rating"].mean() * w1 / 100 + \
           dataframe.loc[(dataframe["Progress"] > 10) & (dataframe["Progress"] <= 45), "Rating"].mean() * w2 / 100 + \
           dataframe.loc[(dataframe["Progress"] > 45) & (dataframe["Progress"] <= 75), "Rating"].mean() * w3 / 100 + \
           dataframe.loc[(dataframe["Progress"] > 75), "Rating"].mean() * w4 / 100
user_based_weighted_average(df, 20, 24, 26, 30)


4.803286469062915

### Weighted Rating (Ağırlıklı Derecelendirme)

- Bu bölümde hem zaman bağımlı hem de kullanıcı bağımlı değerlendirme yapacağız. 

In [65]:
def course_weighted_rating(dataframe, time_w=50, user_w=50):
    return time_based_weighted_average(dataframe) * time_w / 100 + user_based_weighted_average(dataframe) * user_w / 100 
course_weighted_rating(df)
#yani time a ve user a aynı ağırlığı verirsek 4.7826 olarak ortalama puan çıkmaktdır. 

4.782641693469868

In [66]:
# yukardaki işlemi ağırlıkları değiştirerek yapabiliriz. Yani, %40 zamandan %60 user dan gelsin istersek;
def course_weighted_rating(dataframe, time_w=40, user_w=60):
    return time_based_weighted_average(dataframe) * time_w / 100 + user_based_weighted_average(dataframe) * user_w / 100 
course_weighted_rating(df)
#yani timee'a %40 ve user'a %60 ağırlık verdiğimiz zaman 4.7816 olarak ortalama puan çıkmaktdır. 

4.786164895710403